# Preparations

## Notebook extensions

In [1]:
%%javascript
require(['base/js/utils'],
function(utils) {
    utils.load_extensions('IPython-notebook-extensions-3.x/usability/comment-uncomment');
    utils.load_extensions('IPython-notebook-extensions-3.x/usability/dragdrop/main');
});

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

## Imports and configuration

We set the path to the config.cfg file using the environment variable 'JCMPYTHON_CONFIG_FILE'.

In [3]:
import os
os.environ['JCMPYTHON_CONFIG_FILE'] = '/hmi/kme/workspace/scattering_generalized/160719_start/config.cfg'

Now we can import `jcmpython` and `numpy`. Since the parent directory, which contains the jcmpython module, is not automatically in our path, we need to append it before.

In [4]:
import sys
sys.path.append('..')
import jcmpython as jpy
import numpy as np

[INFO] init: This is jcmpython. Starting up.
[INFO] init: Writing logs to file: /hmi/kme/workspace/scattering_generalized/160719_start/examples/logs/160729.log
[DEBUG] init: System info:
[DEBUG] init: 	JCMROOT: /hmi/kme/programs/JCMsuite_3_0_9
[DEBUG] init: 	JCMKERNEL: V3
[DEBUG] init: Initializing resources from configuration.


On import, we receive a some information on the configured logging and JCMsuite version, if the logging level is appropriate. We can get additional info on the version and the license.

In [5]:
jpy.license_info()

[INFO] init: 
[INFO] init: JCMwave(R) FEM-Solver , Version 3.0.9-release
[INFO] init: Modules: Electromagnetics ElectromagneticsTimeDependent ContinuumMechanics HeatConduction
[INFO] init: Buildtag: linux64-2016.06.24.09.46
[INFO] init: 
[INFO] init: *** license status ***
[INFO] init: 
[INFO] init:   JCMROOT: /hmi/kme/programs/JCMsuite_3_0_9
[INFO] init:   Host: nanosippe03
[INFO] init:   License File: /hmi/kme/programs/JCMsuite_3_0_9/license/license.ljcm
[INFO] init:   Server: nanosippe03
[INFO] init:   HW-Adapter: F8-B1-56-D8-95-11
[INFO] init:   Licensed OS: LINUX
[INFO] init:   License period: 2015-06-08 -> 2016-12-31 
[INFO] init:   Number of licenses: 12
[INFO] init: 


# Simulation

## Preparing and configuring the simulation set

We start by creating a `JCMProject`-instance describing the project we'd like to run.  The `mie2D`-project is located in a subdirectory of our *project catalog*, which is configured in the section `Data` under key `projects` in the configuration file. We could also specify an absolute path instead. Since we want to leave the actual project untouched, we specify a `working_dir` into which the project is copied before. `JCMgeo` and the template conversion will be executed in the working directory, for example. If we do not specify a working_dir, a folder called `current_run` will be used in the current directory.

In [6]:
wdir = os.path.abspath('working_dir')
project = jpy.JCMProject('scattering/mie/mie2D', working_dir=wdir)

The JCMProject-instance automatically detected the name of the project file:

In [7]:
project.project_file_name

'mie2D.jcmp'

If it fails to find a proper file or if it finds multiple project files, it raises an Exception. You can specify the project file name manually using the parameter `project_file_name` on initialization.

To run simulations using this project, we create a `SimulationSet` (this could also be single simulation). The keys that are necessary to translate the JCM template files (i.e. the `.jcmp(t)`-files) need to be given as a nested `dict` with keys `constants`, `parameters` and `geometry` in the top level. The values for these keys need to be `dict`s as well, providing all necessary keys for the template translation in total. Their function is as follows:

  - `constants`: can be of any type, but are not stored in the HDF5 store. This is useful for minor parameters, such as the info level in the project, as it does not change the result of the simulation. But it can also be used to pass complicated data, such as material data maps.
  - `parameters`: All parameters that *do not* change the geometry, i.e. do not belong to the `layout.jcmt` template.
  - `geometry`: All parameters that *do* change the geometry, i.e. belong to the `layout.jcmt` template.
  
If a sequence is provided for any of the `parameters` or `geometry` values, loops will be performed (depending on the `combination_mode` of the `SimulationSet`).

In the mie2D project, there is only one parameter: the *radius* of the circle. This parameter changes the geometry! We'd like to scan over different radii and, consequently, provide a numpy.array for it. We leave the other two `dict`s empty.

In [8]:
mie_keys = {'constants' :{},
            'parameters': {},
            'geometry': {'radius':np.linspace(0.3, 0.5, 40)}}

Now, the `SimulationSet` can be initialized.

In [9]:
simuset = jpy.SimulationSet(project, mie_keys)

[DEBUG] root: Removing existing folder /hmi/kme/workspace/scattering_generalized/160719_start/examples/working_dir
[DEBUG] root: Copying project to folder: /hmi/kme/workspace/scattering_generalized/160719_start/examples/working_dir
[INFO] root: Using folder /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729 for data storage.
[DEBUG] root: Initializing the HDF5 store


We are now informed about the directory in which our data is stored as configured in the configuration file and by the `duplicate_path_levels` parameter. The path is also stored in the attribute `storage_dir`. It now contains an `.h5` database file:

In [10]:
os.listdir(simuset.storage_dir)

['result_database.h5', 'simulation000000']

We can now make a schedule for the simulations that we want to perform. This includes that

  1. all parameter combinations are determinded,
  2. the simulations are sorted in a way to have minimal calls of JCMgeo (which can be expensive) and
  3. the database is checked for matching simulations which already have been done.

In our case, the database is still empty and we should end up with 40 simulations, as we specified 40 different radii.

In [11]:
simuset.make_simulation_schedule()

[DEBUG] root: Analyzing loop properties.
[INFO] root: Loops will be done over the following parameter(s): ['radius']
[INFO] root: Total number of simulations: 40
[DEBUG] root: Generating the simulation list.
[DEBUG] root: Sorting the simulations.
[DEBUG] root: Result of the store pre-check: Empty


The store should still be empty at this time:

In [12]:
simuset.is_store_empty()

True

Depending on the configured servers, there might be multiple workstations or queues which can be used by the JCMdaemon to run the simulations. For this 2D project, we can restrict the resources to be only the local computer, i.e. 'localhost'.

In [13]:
simuset.use_only_resources('localhost')

[INFO] root: Restricting resources to: ['localhost']


You can get a list of the currently configured resources that will be used by the SimulationSet using the `get_current_resources`-method.

In [14]:
simuset.get_current_resources()

[Workstation(localhost, M=1, N=1)]

## Computing geometries only

Computing a geometry (i.e. running `jcm.geo`) for a specific simulation of our `SimulationSet` is an easy task now. We only need to call the `compute_geometry` method with the index of the simulation (or the `Simulation` instance itself). We can pass additional keyword arguments to jcm.geo, such as a `jcmt_pattern` if desired.

Let's compute the geometry for the first simulation of our set. The simulation has the following properties:

In [15]:
sim = simuset.simulations[0]
print sim
print 'keys:', sim.keys

Simulation(number=0, status=Pending)
keys: {'radius': 0.29999999999999999}


Now we run jcm.geo for it (we also could have written `simuset.compute_geometry(0)`)

In [16]:
simuset.compute_geometry(sim)

[DEBUG] root: Computing geometry.


The project's working directory now contains a grid file:

In [17]:
os.listdir(simuset.get_project_wdir())

['layout.jcmt',
 'sources.jcm',
 'triangulator.jcm',
 'grid.jcm',
 'materials.jcm',
 'mie2D.jcmp',
 'layout.jcm']

## Running simulations

In [18]:
simuset.run()

[DEBUG] jcmpython.parallelization: Adding Workstation(localhost, M=1, N=1)
[DEBUG] jcmpython.parallelization: ... adding was successful.
[DEBUG] root: Starting to simulate.
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 1 of 40 with jobID 673458234
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 2 of 40 with jobID 2090203248
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 3 of 40 with jobID 609864098
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 4 of 40 with jobID 1925905960
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 5 of 40 with jobID 1199005044
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 6 of 40 with jobID 2124831418
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 7 of 40 with jobID 45207571
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulation 8 of 40 with jobID 1972651129
[DEBUG] root: Computing geometry.
[DEBUG] root: Queued simulati

DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000000/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000001/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000002/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000003/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000004/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000005/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scattering_generalized/160719_start/examples/160729/simulation000006/mie2D.jcmp
DAEMON.WAIT:  Failed: /net/group/kme-data/simulations/scatteri

AttributeError: 'SimulationSet' object has no attribute 'cleanMode'